In [1]:
import pandas as pd
from collections import Counter
import math
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pandas as pd


c:\Users\Lasse\anaconda3\envs\bachelorproject\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# Load real and fake BuzzFeed datasets
df_real = pd.read_csv('../data/raw/BuzzFeed_real_news_content.csv')
df_fake = pd.read_csv('../data/raw/BuzzFeed_fake_news_content.csv')

In [ ]:

# Concatenate them into a full dataset
df_full = pd.concat([df_real, df_fake], ignore_index=True)

# Append news_id from PolitiFactNews.txt
with open('../data/raw/BuzzFeedNews.txt', 'r') as file:
    news_ids = [line.strip() for line in file]

df_full["news_id"] = news_ids

# Load train and test id's
with open('../data/processed/FakeNewsNet/trainIds.txt', 'r') as file:
    train_values = [line.strip() for line in file]
with open('../data/processed/FakeNewsNet/testIds.txt', 'r') as file:
    test_values = [line.strip() for line in file]

# Create train and test data sets
df_train = df_full[df_full['news_id'].isin(train_values)]
df_test = df_full[df_full['news_id'].isin(test_values)]

df_train["full_text"] = df_train["title"] + " " + df_train["text"]
df_test["full_text"] = df_test["title"] + " " + df_test["text"]

# Create a label column stating whether the news is fake or real based on the news_id column
df_train["label"] = df_train["news_id"].apply(lambda x: "fake" if x.split("_")[1].lower() == "fake" else "real")
df_test["label"] = df_test["news_id"].apply(lambda x: "fake" if x.split("_")[1].lower() == "fake" else "real")
